In [1]:
from typing import Set, List, Tuple, Dict, Iterable
import re
from typing import NamedTuple
import math
from collections import defaultdict

In [2]:
# Токенизация сообщений
def tokenize(text: str) -> Set[str]:
    text = text.lower()
    all_words = re.findall("[a-z0-9']+", text) # Извлечь слова
    return set(all_words)   # Удалить повторы

In [3]:
class Message(NamedTuple):
    text: str
    is_spam: bool

In [4]:
class NaiveBayesClassifier:
    def __init__(self, k: float = 0.5) -> None:
        self.k = k  # Сглаживающий фильтр

        self.tokens: Set[str] = set()
        self.token_spam_counts: Dict[str, int] = defaultdict(int)
        self.token_ham_counts: Dict[str, int] = defaultdict(int)
        self.spam_messages = self.ham_messages = 0
    def train(self, messages: Iterable[Message]) -> None:
        for message in messages:
            # Увеличение количества сообщений
            if message.is_spam:
                self.spam_messages += 1
            else:
                self.ham_messages += 1

            # Увеличение количества появлений слов
            for token in tokenize(message.text):
                self.tokens.add(token)
                if message.is_spam:
                    self.token_spam_counts[token] += 1
                else:
                    self.token_ham_counts[token] += 1
    def _probabilities(self, token: str) -> Tuple[float, float]:
        """Возвращает P(лексема | спам) и P(лексема | неспам)"""
        spam = self.token_spam_counts[token]
        ham = self.token_ham_counts[token]

        p_token_spam = (spam + self.k) / (self.spam_messages + 2 * self.k)
        p_token_ham = (ham + self.k) / (self.ham_messages + 2 * self.k)

        return p_token_spam, p_token_ham
    def predict(self, text: str) -> float:
        text_tokens = tokenize(text)
        log_prob_if_spam = log_prob_if_ham = 0.0

        # Перебрать все слова в лексиконе
        for token in self.tokens:
            prob_if_spam, prob_if_ham = self._probabilities(token)
            # Если лексема появляется в сообщении, то добавить лог. вероятность ее встретить
            if token in text_tokens:
                log_prob_if_spam += math.log(prob_if_spam)
                log_prob_if_ham += math.log(prob_if_ham)
            # В противном случае добавить лог. вероятность ее не встретить
            else:
                log_prob_if_spam += math.log(1.0 - prob_if_spam)
                log_prob_if_ham += math.log(1.0 - prob_if_ham)

        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_ham = math.exp(log_prob_if_ham)
        return prob_if_spam / (prob_if_spam + prob_if_ham)

In [5]:
messages = [Message("spam rules", is_spam=True),
            Message("ham rules", is_spam=False),
            Message("hello ham", is_spam=False)]
model = NaiveBayesClassifier(k = 0.5)
model.train(messages)

In [7]:
# Предсказание классификатора
text = "hello spam"
model_prediction = model.predict(text)
print(model_prediction)

0.8350515463917526


In [9]:
# Предсказание по формуле Байеса
probs_if_spam = [
    (1+0.5) / (1 + 2 * 0.5),
    1 - (0 + 0.5) / (1 + 2 * 0.5),
    1 - (1 + 0.5) / (1 + 2 * 0.5),
    (0 + 0.5) / (1 + 2 * 0.5)
]

probs_if_ham = [
    (0 + 0.5) / (2 + 2 * 0.5),
    1 - (2 + 0.5) / (2 + 2 * 0.5),
    1 - (1 + 0.5) / (2 + 2 * 0.5),
    (1 + 0.5) / (2 + 2 * 0.5)
]

p_if_spam = math.exp(sum(math.log(p) for p in probs_if_spam))
p_if_ham = math.exp(sum(math.log(p) for p in probs_if_ham))
print(p_if_spam / (p_if_spam + p_if_ham))

0.8350515463917525


<p>Как видно, результаты предсказаниz по формуле Байеса и результаты обученной модели совпадают</p>